In [31]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
widths = [
    4,
    2, 
    2,
    3,
    2,
    1,
    1,
    1,
    2,
    8
]

first_load = pd.read_fwf("us.1969_2018.19ages.txt", widths=widths, low_memory= True)
first_load

,1969,AL,01,001,99,1,9,1.1,00,00000159
0,1969,AL,1,1,99,1,9,1,1,657
1,1969,AL,1,1,99,1,9,1,2,1137
2,1969,AL,1,1,99,1,9,1,3,956
3,1969,AL,1,1,99,1,9,1,4,721
4,1969,AL,1,1,99,1,9,1,5,424
...,...,...,...,...,...,...,...,...,...,...
14862468,2018,WY,56,45,99,3,9,2,14,8
14862469,2018,WY,56,45,99,3,9,2,15,3
14862470,2018,WY,56,45,99,3,9,2,16,1
14862471,2018,WY,56,45,99,3,9,2,17,2


In [95]:
first_load.to_csv("c", index=False)

In [96]:
#yelp_nv = yelp_df2[yelp_df2["state_cd"]== "NV"]

year_1990 = first_load[first_load["1969"]>1989]
year_1990.head()

,1969,AL,01,001,99,1,9,1.1,00,00000159
5916602,1990,AL,1,1,99,1,9,1,0,241
5916603,1990,AL,1,1,99,1,9,1,1,828
5916604,1990,AL,1,1,99,1,9,1,2,1102
5916605,1990,AL,1,1,99,1,9,1,3,1153
5916606,1990,AL,1,1,99,1,9,1,4,1050


In [97]:
year_1990.to_csv("year_1990.csv", index=False)

In [86]:
census_load = pd.read_csv("year_1990.csv", names=["year", "state","State_FIPS","County_FIPS","Registry", "Race", "Origin", "Sex","Age","Population"])
census_load.head()

,year,state,State_FIPS,County_FIPS,Registry,Race,Origin,Sex,Age,Population
0,1969,AL,1,1,99,1,9,1.1,0,159
1,1990,AL,1,1,99,1,9,1.0,0,241
2,1990,AL,1,1,99,1,9,1.0,1,828
3,1990,AL,1,1,99,1,9,1.0,2,1102
4,1990,AL,1,1,99,1,9,1.0,3,1153


In [87]:
census_load = census_load.drop([0])
census_load.head()

,year,state,State_FIPS,County_FIPS,Registry,Race,Origin,Sex,Age,Population
1,1990,AL,1,1,99,1,9,1.0,0,241
2,1990,AL,1,1,99,1,9,1.0,1,828
3,1990,AL,1,1,99,1,9,1.0,2,1102
4,1990,AL,1,1,99,1,9,1.0,3,1153
5,1990,AL,1,1,99,1,9,1.0,4,1050


In [88]:
census_groupby = census_load.groupby(["year", "Race","Origin", "Sex", "Age"]).agg({"Population":"sum"}).reset_index()
census_groupby.head()

,year,Race,Origin,Sex,Age,Population
0,1990,1,9,1.0,0,1624007
1,1990,1,9,1.0,1,6079759
2,1990,1,9,1.0,2,7453852
3,1990,1,9,1.0,3,7079291
4,1990,1,9,1.0,4,7323128


In [89]:
census_groupby.isnull().values.any()

False

In [90]:
census_groupby.isnull().sum()

year          0
Race          0
Origin        0
Sex           0
Age           0
Population    0
dtype: int64

In [91]:
#Check rows for dup
boolean = census_groupby.duplicated().any() 
boolean

False

In [92]:
#clean up values in rows

census_groupby["Sex"] = census_groupby.Sex.replace([1.0,2.0],["Male","Female"])
census_groupby["Race"] = census_groupby.Race.replace([1,2,3,4],["White","Black","American Indian/Alaska Native","Asian or Pacific Islander"])
census_groupby["Origin"] = census_groupby.Origin.replace([0,1,9],["Non-Hispanic","Hispanic","Not applicable"])
census_groupby["Age"] = census_groupby.Age.replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
["00-04","00-04","05-09","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45-49","50-54","55-59","60-64","65-69","70-74","75-79","80-84","85+"])
census_groupby

,year,Race,Origin,Sex,Age,Population
0,1990,White,Not applicable,Male,00-04,1624007
1,1990,White,Not applicable,Male,00-04,6079759
2,1990,White,Not applicable,Male,05-09,7453852
3,1990,White,Not applicable,Male,10-14,7079291
4,1990,White,Not applicable,Male,15-19,7323128
...,...,...,...,...,...,...
3301,2018,American Indian/Alaska Native,Not applicable,Female,65-69,583573
3302,2018,American Indian/Alaska Native,Not applicable,Female,70-74,422050
3303,2018,American Indian/Alaska Native,Not applicable,Female,75-79,282236
3304,2018,American Indian/Alaska Native,Not applicable,Female,80-84,189544


In [93]:
census_final = census_groupby.groupby(["year", "Race","Origin", "Sex", "Age"]).agg({"Population":"sum"}).reset_index()
census_final.head()

,year,Race,Origin,Sex,Age,Population
0,1990,American Indian/Alaska Native,Not applicable,Female,00-04,426930
1,1990,American Indian/Alaska Native,Not applicable,Female,05-09,410011
2,1990,American Indian/Alaska Native,Not applicable,Female,10-14,381105
3,1990,American Indian/Alaska Native,Not applicable,Female,15-19,395579
4,1990,American Indian/Alaska Native,Not applicable,Female,20-24,413921


In [94]:
census_final.to_csv("census_final.csv", index=False)